In [1]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver  # 라이브러리(모듈) 가져오라
from selenium.webdriver import ActionChains as AC
import chromedriver_autoinstaller
from tqdm import tqdm
from tqdm.notebook import tqdm
import re
from time import sleep
import time
from tqdm import tqdm_notebook

from konlpy.tag import Hannanum
import requests
from bs4 import BeautifulSoup
from konlpy.tag import Twitter
from collections import Counter
# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# 추천시스템
import surprise
from surprise import SVD
from surprise import NMF
from surprise import KNNBasic

In [2]:
data = pd.read_excel('siksin_2.xlsx')

In [3]:
data

,Unnamed: 0,닉네임,평가,평점,가게명,지역
0,273,!날아라슈퍼맨!,매운족발 그냥 깔끔합니다.,3.0,꿀벅지족발,"서울-강북, 남산/후암동"
1,976,!날아라슈퍼맨!,한우 갈비살 양념구이 먹엇는데 석쇠에 빨리빨리 구워서 먹는데 여기특이한게 고추장에 ...,4.0,영양숯불갈비,"경북, 경주시"
2,2438,!날아라슈퍼맨!,닭불고기 정말 만족해요,3.0,무등산닭불고기,"광주, 서구-금호/풍암"
3,3341,!날아라슈퍼맨!,제주도에서 먹은 고기국수 만큼 괜찮았어요 ㅎㅎ,3.0,아랑졸디,"경기, 구리"
4,3467,!날아라슈퍼맨!,누룽지백숙!! 백숙국물에 누룽지를 넣어서 나오는데 담백하고 고소하고 닭백숙이 되게 ...,3.0,산촌,"경기, 의왕/백운호수"
...,...,...,...,...,...,...
5340,6024,힙통령,만족스러워요,2.5,내림손삼계탕,"서울-강북, 명동"
5341,628,힙합간지녀,돼지갈비도 인기 많아요,3.0,청석정육식당,"경기, 광주시"
5342,767,힙합간지녀,등심 진짜 맛있어요,3.0,창신식육식당,"경북, 영주"
5343,2214,힙합간지녀,고기를 직접 골라서 먹을 수 있는게 정말 맘에 들어요 ㅋㅋ 한우랑 키조개랑 같이 먹...,3.0,탐마루장흥한우,"전남, 장흥"


In [4]:
print(data.shape)

(5345, 6)


In [5]:
df = data[['닉네임', '가게명', '평점']]
df['평점'] = df['평점'].astype(int)
df

,닉네임,가게명,평점
0,!날아라슈퍼맨!,꿀벅지족발,3
1,!날아라슈퍼맨!,영양숯불갈비,4
2,!날아라슈퍼맨!,무등산닭불고기,3
3,!날아라슈퍼맨!,아랑졸디,3
4,!날아라슈퍼맨!,산촌,3
...,...,...,...
5340,힙통령,내림손삼계탕,2
5341,힙합간지녀,청석정육식당,3
5342,힙합간지녀,창신식육식당,3
5343,힙합간지녀,탐마루장흥한우,3


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5345 entries, 0 to 5344
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   닉네임     5345 non-null   object
 1   가게명     5345 non-null   object
 2   평점      5345 non-null   int32 
dtypes: int32(1), object(2)
memory usage: 104.5+ KB


In [7]:
# 판다스를 딕셔너리로 만드는 함수
def recur_dictify(frame):
    if len(frame.columns) == 1:
        if frame.values.size == 1: return frame.values[0][0]
        return frame.values.squeeze()
    grouped = frame.groupby(frame.columns[0])
    d = {k: recur_dictify(g.iloc[:,1:]) for k,g in grouped}
    return d

In [8]:
name_list = []      # 사용자 목록을 담을 리스트   # 중복 가능
food_set = set()  

In [9]:
df_dict = recur_dictify(df)

In [10]:
df_dict

{'!날아라슈퍼맨!': {'꿀벅지족발': 3,
  '남자곱창': 3,
  '닭발의지존': 3,
  '무등산닭불고기': 3,
  '산촌': 3,
  '아랑졸디': 3,
  '영양숯불갈비': 4},
 '#Dali_Mom': {'대가집보쌈': 3,
  '명동본가닭갈비': 3,
  '센치한닭': 2,
  '아는오빠 고기집': 2,
  '역전한우숯불식당': 2,
  '원조한방삼계탕': 3,
  '청운갈비': 3,
  '한우농장정육식당': 3},
 '#_허니잼': {'곱창고': 3,
  '도란도란': 3,
  '배서방족발집': 3,
  '원도매불고기': 3,
  '원조숯불닭불고기': 3,
  '원평지평선청보리한우촌': 3,
  '화산불고기': 2,
  '황금수산닭갈비': 3},
 '#돼랑이라그램': {'고수찜닭': 2,
  '꼭그닭 강남점': 3,
  '동적불고기': 2,
  '리강왕곱창': 3,
  '맛나숯불바비큐치킨': 3,
  '백억하누본점': 3,
  '오성회관': 3,
  '지호한방삼계탕 여수점': 3,
  '철길왕갈비살': 3,
  '청자골': 3,
  '초당참숯화로구이': 3,
  '큰바위가든': 3},
 '#먹방': {'금수산식당': 3,
  '논골집': 3,
  '복수청정한우': 3,
  '샘밭숯불닭갈비': 3,
  '온천골 본점': 3,
  '최고집보쌈': 3},
 '#혼자밥': {'광세족발': 3,
  '구능촌': 3,
  '꿀벅지족발': 3,
  '대한닭발 정릉점': 3,
  '동진식당': 2,
  '마약생고기': 3,
  '새대구숯불구이': 3,
  '승일식당': 3,
  '양동통닭': 3,
  '이조보쌈': 3,
  '장충동웰빙족발': 3,
  '족발의 명수': 2,
  '춘천고려닭갈비': 3,
  '할머니 보쌈족발 김포장기점': 2,
  '히로아키': 2},
 '*모래알은반짝*': {'배가족발': 4,
  '보배곱창': 4,
  '삼명숯불가든': 3,
  '삼목등심': 3,
  '삼우갈비': 4,
  '삽교원조두리곱창': 3,
  '숯가마바베큐

In [11]:
# 사용자 수 만큼 반복한다.
for user_key in df_dict:  # user_key : 각각의 유저이름
#     print(user_key)
    name_list.append(user_key)
    
    # 현재 사용자가 본 화장품 목록을 set에 담는다.
    for food_key in df_dict[user_key] :   # 각각의 유저이름에 해당하는 각각의 제품이름
        # print(user_key, ":", cos_key)
        food_set.add(food_key)  

In [12]:
food_set

{'개성삼계탕',
 '부흥한우실비식당',
 '옛장터 숯불갈비',
 '사이치킨',
 '서명작',
 '보조락',
 '연춘참숯석쇠구이',
 '신마포갈매기',
 '국수의전설',
 '대궁삼계탕',
 '북청생고기',
 '한우뭉치',
 '내당한우',
 '동래삼계탕',
 '횡성한우곰탕',
 '다하누등심플러스',
 '수원서서갈비',
 '지인고기',
 '농부의 한우',
 '옹기종기',
 '철마한우직판장',
 '철마한우 우송',
 '우향안동한우식육식당 본점',
 '경주식당',
 '족발쌀롱',
 '육회의갑',
 '미쓰족발 서면점',
 '춘천닭갈비 산촌막국수',
 '솥뚜껑닭갈비',
 '송암가든',
 '육사구이',
 '곱창고',
 '고기의 잔치',
 '연기숯불갈비',
 '고기위에상추쌈',
 '천지유',
 '동기간',
 '일등갈비',
 '고가길구공탄',
 '마고닭',
 '돈바른 수제갈비',
 '본가한방삼계탕',
 '예이제갈비',
 '강수돌곱창',
 '꼬꼬사우나찜닭 삼산점',
 '양가곱창',
 '왜불러곱창',
 '옥거리',
 '한우천국',
 '범근네 헛개나무족발',
 '은하갈비',
 '곱창고 방이점',
 '꼴통치킨',
 '한우리한정식',
 '마약생고기',
 'OK장충동왕족발보쌈',
 '아뜨리에',
 '봉남이네',
 '서경도락',
 '천하불닭발',
 '행님육회',
 '무진장',
 '태극식당',
 '일월산생고기직화구이',
 '파전닭갈비',
 '광양불고기 대치점',
 '리북집',
 '할머니 보쌈족발 김포장기점',
 '대관령한우프라자',
 '남양주한우마을',
 '마포한우곱창',
 '구움',
 '밴댕이소갈딱지삼겹살강가네',
 '배떼기곱창',
 '방일해장국본점',
 '월정리갈비밥 서귀포점',
 '락송정',
 '오죽헌가든',
 '이동부산갈비',
 '마포곱창타운',
 '닭발예찬',
 '청산곱창',
 '원조북성로우동불고기',
 '돌다리집',
 '장수한우명품관',
 '통일집',
 '춘천닭갈비',
 '그램그램 구포점',
 '백제삼계탕',
 '화롯가',
 '최신족발',
 '지기네불닭발',
 '북막골',
 '달

In [13]:
food_list = list(food_set)

print(len(food_list))
food_list

1982


['개성삼계탕',
 '부흥한우실비식당',
 '옛장터 숯불갈비',
 '사이치킨',
 '서명작',
 '보조락',
 '연춘참숯석쇠구이',
 '신마포갈매기',
 '국수의전설',
 '대궁삼계탕',
 '북청생고기',
 '한우뭉치',
 '내당한우',
 '동래삼계탕',
 '횡성한우곰탕',
 '다하누등심플러스',
 '수원서서갈비',
 '지인고기',
 '농부의 한우',
 '옹기종기',
 '철마한우직판장',
 '철마한우 우송',
 '우향안동한우식육식당 본점',
 '경주식당',
 '족발쌀롱',
 '육회의갑',
 '미쓰족발 서면점',
 '춘천닭갈비 산촌막국수',
 '솥뚜껑닭갈비',
 '송암가든',
 '육사구이',
 '곱창고',
 '고기의 잔치',
 '연기숯불갈비',
 '고기위에상추쌈',
 '천지유',
 '동기간',
 '일등갈비',
 '고가길구공탄',
 '마고닭',
 '돈바른 수제갈비',
 '본가한방삼계탕',
 '예이제갈비',
 '강수돌곱창',
 '꼬꼬사우나찜닭 삼산점',
 '양가곱창',
 '왜불러곱창',
 '옥거리',
 '한우천국',
 '범근네 헛개나무족발',
 '은하갈비',
 '곱창고 방이점',
 '꼴통치킨',
 '한우리한정식',
 '마약생고기',
 'OK장충동왕족발보쌈',
 '아뜨리에',
 '봉남이네',
 '서경도락',
 '천하불닭발',
 '행님육회',
 '무진장',
 '태극식당',
 '일월산생고기직화구이',
 '파전닭갈비',
 '광양불고기 대치점',
 '리북집',
 '할머니 보쌈족발 김포장기점',
 '대관령한우프라자',
 '남양주한우마을',
 '마포한우곱창',
 '구움',
 '밴댕이소갈딱지삼겹살강가네',
 '배떼기곱창',
 '방일해장국본점',
 '월정리갈비밥 서귀포점',
 '락송정',
 '오죽헌가든',
 '이동부산갈비',
 '마포곱창타운',
 '닭발예찬',
 '청산곱창',
 '원조북성로우동불고기',
 '돌다리집',
 '장수한우명품관',
 '통일집',
 '춘천닭갈비',
 '그램그램 구포점',
 '백제삼계탕',
 '화롯가',
 '최신족발',
 '지기네불닭발',
 '북막골',
 '달

In [14]:
print(name_list)

['!날아라슈퍼맨!', '#Dali_Mom', '#_허니잼', '#돼랑이라그램', '#먹방', '#혼자밥', '*모래알은반짝*', '*뽀로로*', '104731', '10시25분', '116767', '121579', '123545', '125097', '125843', '128632', '129355', '13', '131199', '134813', '1373', '137428', '1433', '15', '151442', '152545', '1675', '168032', '16905', '201820', '202동202호', '211655', '248023', '268314', '274412', '275938', '283938', '2momo_cine\nSIKSINER', '2차원개그', '3', '303206', '308238', '311885', '316967', '335821', '341413', '345002', '359487', '359613', '360591', '361479', '365일라땡', '371596', '379379', '383172', '38725', '387376', '39', '392898', '394364', '405375', '405626\nSIKSINER', '410601', '410648', '418166', '418190', '424364', '427048', '43016', '430832', '435817', '436666', '441023', '450824', '454297', '454298', '454300', '454303', '454304', '454305', '454306', '454308', '454309', '454310', '454311', '454316', '454317', '454318', '454319', '454321', '454326', '454331', '454346', '454352', '454357', '454358', '454359', '454361', '454368', '454370',

In [15]:
# 학습할 데이터를 준비한다.
rating_dic = {
    '닉네임' : [],
    '가게명' : [],
    '평점' : []
    }

In [16]:
rating_dic

{'닉네임': [], '가게명': [], '평점': []}

In [17]:
# user 수만큼 반복
user_list = []
for user_key in df_dict :  # user_key : 각각의 유저이름
    user_list.append(user_key)
    # 해당 사용자가 본 화장품 수 만큼 반복
    for food_key in df_dict[user_key] :

        # 사용자 인덱스 번호를 추출한다.
        a1 = name_list.index(user_key)

        # 화장품 인덱스 번호를 추출한다.
        a2 = food_list.index(food_key)

        # 평점을 가져온다.
        a3 = df_dict[user_key][food_key]

        # 딕셔너리에 담는다.
        rating_dic['닉네임'].append(a1)
        rating_dic['가게명'].append(a2)
        rating_dic['평점'].append(a3)

In [18]:
rating_dic

{'닉네임': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  3,
  4,
  4,
  4,
  4,
  4,
  4,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  6,
  7,
  7,
  7,
  8,
  8,
  8,
  8,
  8,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  9,
  10,
  11,
  11,
  11,
  12,
  12,
  12,
  12,
  12,
  12,
  13,
  13,
  13,
  13,
  13,
  13,
  13,
  14,
  14,
  14,
  14,
  14,
  14,
  15,
  15,
  16,
  16,
  17,
  17,
  18,
  18,
  18,
  18,
  19,
  19,
  19,
  20,
  20,
  20,
  20,
  20,
  21,
  22,
  22,
  23,
  23,
  23,
  24,
  24,
  25,
  25,
  25,
  25,
  25,
  26,
  26,
  26,
  26,
  26,
  26,
  26,
  26,
  27,
  27,
  27,
  27,
  28,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  29,
  30,
  30,
  30,
  30,
  30,
  30,
  31,
  31,
  32,
  32,
  32,
  32,
  33,
  33,
  33,
  3

In [19]:
df_index = pd.DataFrame(rating_dic)
df_index

,닉네임,가게명,평점
0,0,1961,3
1,0,1045,3
2,0,707,3
3,0,1118,3
4,0,1261,3
...,...,...,...
5340,928,578,3
5341,929,919,3
5342,929,1534,3
5343,929,1182,3


In [20]:
df_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5345 entries, 0 to 5344
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   닉네임     5345 non-null   int64
 1   가게명     5345 non-null   int64
 2   평점      5345 non-null   int64
dtypes: int64(3)
memory usage: 125.4 KB


In [21]:
import surprise
# 데이터를 읽어들이는 객체 생성 (rating_scale : 평점의 범위)
reader = surprise.Reader(rating_scale = (0, 5))
reader

In [22]:
col_list = ['닉네임', '가게명', '평점']
data = surprise.Dataset.load_from_df(df_index[col_list], reader)

In [23]:
print(col_list)
print(data)

['닉네임', '가게명', '평점']


In [24]:
# 학습한다.
trainset = data.build_full_trainset()
option = {'name' : 'pearson'}
algo = surprise.KNNBasic(sim_options=option)

algo.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [25]:
from surprise import SVD
from surprise.model_selection import cross_validate
from surprise import NMF
from surprise import KNNBasic

In [26]:
cross_validate(algo, data)["test_mae"].mean()

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.


0.6035660315205519

In [27]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    0.7352  0.7948  0.8544  0.7921  0.779

{'test_rmse': array([0.73516262, 0.79483016, 0.85438063, 0.79211054, 0.77985191,
        0.76968938, 0.76240081, 0.82194425, 0.79763152, 0.79630224]),
 'test_mae': array([0.5544741 , 0.59875931, 0.65084132, 0.59729406, 0.59099514,
        0.58482109, 0.57842655, 0.62586564, 0.59528988, 0.60926158]),
 'fit_time': (0.09401845932006836,
  0.11868500709533691,
  0.09169816970825195,
  0.08022665977478027,
  0.11871623992919922,
  0.08474063873291016,
  0.09975767135620117,
  0.1156930923461914,
  0.09771442413330078,
  0.11022663116455078),
 'test_time': (0.013116598129272461,
  0.007036685943603516,
  0.006979465484619141,
  0.010934591293334961,
  0.014991521835327148,
  0.013961315155029297,
  0.008962154388427734,
  0.021940946578979492,
  0.019002437591552734,
  0.015956640243530273)}

In [28]:
for i in tqdm_notebook(range(len(name_list))):
    name_list[i] = str(name_list[i])

In [30]:
# "사용자1"의 음식점을 추천받는다.
who = input('닉네임을 입력해주세요: ')
print('\n')

index = name_list.index(who)
print('user_index: ',index)
print('\n')

result = algo.get_neighbors(index, k=5)
print('당신과 비슷한 사람은? : ', result)
print('\n')

# user 귯니스에 대해 화장품을 추천한다.
print('당신에게 추천드리는 맛집 : ', '\n')

for r1 in result :
    max_rating=data.df[data.df["닉네임"]==r1]["평점"].max()
    cos_id=data.df[(data.df["평점"]==max_rating)&(data.df["닉네임"]==r1)]["가게명"].values
    
    for cos_item in cos_id:
        print(food_list[cos_item])

닉네임을 입력해주세요: 힙합간지녀


user_index:  929




IndexError: index 929 is out of bounds for axis 0 with size 925